### ☕ Coffee Quality database from CQI ☕

These data contain reviews of 1312 arabica and 28 robusta coffee beans from the Coffee Quality Institute's trained reviewers.


_______
Importando bibliotecas
_____

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

_______
Importando o dataset
___

In [17]:
df_coffee = pd.read_csv('merged_data_cleaned.csv')
df_coffee.shape

(1339, 44)

In [ ]:
df_coffee.head(3)

In [ ]:
df_coffee.columns

In [28]:
cols = [0,2,4,5,6,7,11,37,38,39]

df_coffee.drop(df_coffee.columns[cols], axis = 1, inplace = True)

In [29]:
df_coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Species               1339 non-null   object 
 1   Country.of.Origin     1338 non-null   object 
 2   Company               1130 non-null   object 
 3   Altitude              1113 non-null   object 
 4   Region                1280 non-null   object 
 5   Number.of.Bags        1339 non-null   int64  
 6   Bag.Weight            1339 non-null   object 
 7   In.Country.Partner    1339 non-null   object 
 8   Harvest.Year          1292 non-null   object 
 9   Grading.Date          1339 non-null   object 
 10  Owner.1               1332 non-null   object 
 11  Variety               1113 non-null   object 
 12  Processing.Method     1169 non-null   object 
 13  Aroma                 1339 non-null   float64
 14  Flavor                1339 non-null   float64
 15  Aftertaste           

In [31]:
df_coffee.describe().round(3)
#Altitude maxima esta bem alta - outlier

,Number.of.Bags,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points,Moisture,Category.One.Defects,Quakers,Category.Two.Defects,altitude_low_meters,altitude_high_meters,altitude_mean_meters
count,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1339.000,1338.000,1339.000,1109.000,1109.000,1109.000
mean,154.183,7.567,7.520,7.401,7.536,7.517,7.518,9.835,9.835,9.857,7.503,82.090,0.088,0.479,0.173,3.556,1750.713,1799.348,1775.031
std,129.987,0.378,0.398,0.404,0.380,0.370,0.409,0.555,0.764,0.616,0.473,3.501,0.048,2.550,0.832,5.313,8669.441,8668.806,8668.626
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,1.000
25%,14.000,7.420,7.330,7.250,7.330,7.330,7.330,10.000,10.000,10.000,7.250,81.080,0.090,0.000,0.000,0.000,1100.000,1100.000,1100.000
50%,175.000,7.580,7.580,7.420,7.580,7.500,7.500,10.000,10.000,10.000,7.500,82.500,0.110,0.000,0.000,2.000,1310.640,1350.000,1310.640
75%,275.000,7.750,7.750,7.580,7.750,7.670,7.750,10.000,10.000,10.000,7.750,83.670,0.120,0.000,0.000,4.000,1600.000,1650.000,1600.000
max,1062.000,8.750,8.830,8.670,8.750,8.580,8.750,10.000,10.000,10.000,10.000,90.580,0.280,63.000,11.000,55.000,190164.000,190164.000,190164.000
